# Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

C:\Users\Savannah\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Savannah\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Savannah\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Savannah\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWa

# Load tag data

In [2]:
path = "../data/tags.xlsx"
transcriber_df = pd.read_excel(path, converters={'space':str, 'nospace':str}, sheet_name="ZooniverseTags")
expert_df = pd.read_excel(path, converters={'space':str, 'nospace':str}, sheet_name="ExpertTags")

expert_tags = expert_df['nospace'].tolist()
transcriber_tags = transcriber_df['nospace'].tolist()

# Get tag embeddings

In [3]:
transcriber_embed = model.encode(transcriber_df['space'].tolist())
expert_embed = model.encode(expert_df['space'].tolist())

# Compare transcriber tag embeddings to expert tag embeddings

In [22]:
def compare_all():
    distances = scipy.spatial.distance.cdist(transcriber_embed, expert_embed, "cosine")
    ranking = [sorted(zip(range(len(distances[i])), distances[i]), key=lambda x: x[1]) for i in range(len(transcriber_df))]
    return [{
        "index": i,
        "transcriber_tag": transcriber_tags[i],
        "expert_tag_1": expert_tags[ranking[i][0][0]],
        "score_1": 1-ranking[i][0][1],
        "expert_tag_2": expert_tags[ranking[i][1][0]],
        "score_2": 1-ranking[i][1][1],
        "expert_tag_3": expert_tags[ranking[i][2][0]],
        "score_3": 1-ranking[i][2][1],}
        for i in range(len(transcriber_df))]

match_df = pd.DataFrame(compare_all())

# Display 50 least-matched expert tags

In [14]:
match_df['expert_tag_1'].value_counts().sort_values().head(50)

heavybombardmentgroups    1
attitudetowardgermany     1
confidenceinship          1
opinionofallies           1
germanradio               1
japan                     2
moraleprogram             2
newsmaps                  2
civilians                 2
bbc                       2
sex                       2
italy                     2
prideinoutfit             2
germanclothing            2
mess                      2
sexcontact                2
postwaropportunities      3
germans                   3
ratings                   3
workingwithcivilians      3
aficourses                3
magazines                 3
pacifictoughness          3
redcross                  3
usoclubs                  3
localcivilians            3
krations                  3
attitudetowardengland     4
entalhealth               4
strategicbombing          4
artillery                 4
japanese                  4
treatmentofpows           4
hospitalfacilities        4
politicalviews            4
pxs                 

# Top transcriber tag to add to expert tags

In [26]:
n = 5 # number of top tags to find

best_indices = [-1]*n

for i in range(n):
    start_time = time.perf_counter()
    
    best_sum = float('inf')
    best_index = -1
    
    for j in range(len(transcriber_df)):
        if j in best_indices:
            continue
            
        expert_tags = np.append(expert_tags, transcriber_tags[j])
        expert_embed = np.vstack((expert_embed, transcriber_embed[j]))
        
        results = compare_all()
        
        sum_cs = sum([results[k]['score_1'] for k in range(len(results)) if k!=j and k not in best_indices])
        
        if sum_cs < best_sum:
            best_sum = sum_cs
            best_index = j
        
        expert_tags = np.delete(expert_tags, len(expert_tags)-1,0)
        expert_embed = np.delete(expert_embed, len(expert_embed)-1,0)
        
        if j%100==0:
            print("Progress:",i,j)
            
    print("TAG FOUND:",transcriber_tags[best_index])
    best_indices[i] = best_index
    expert_tags = np.append(expert_tags, transcriber_tags[best_index])
    expert_embed = np.vstack((expert_embed, transcriber_embed[best_index]))
    
    stop_time = time.perf_counter()
    print("Time for tag",i+1,":",stop_time-start_time)

Progress: 0 0
Progress: 0 100
Progress: 0 200
Progress: 0 300
Progress: 0 400
Progress: 0 500
Progress: 0 600
Progress: 0 700
Progress: 0 800
Progress: 0 900
Progress: 0 1000
Progress: 0 1100
Progress: 0 1200
Progress: 0 1300
Progress: 0 1400
Progress: 0 1500
Progress: 0 1600
Progress: 0 1700
Progress: 0 1800
Progress: 0 1900
Progress: 0 2000
Progress: 0 2100
Progress: 0 2200
Progress: 0 2300
Progress: 0 2400
Progress: 0 2500
Progress: 0 2600
Progress: 0 2700
Progress: 0 2800
Progress: 0 2900
Progress: 0 3000
Progress: 0 3100
Progress: 0 3200
Progress: 0 3300
Progress: 0 3400
Progress: 0 3500
Progress: 0 3600
Progress: 0 3700
Progress: 0 3800
Progress: 0 3900
TAG FOUND: 11airborn
Time for tag 1 : 13165.266901899999
Progress: 1 100
Progress: 1 200
Progress: 1 300
Progress: 1 400
Progress: 1 500
Progress: 1 600
Progress: 1 700
Progress: 1 800
Progress: 1 900
Progress: 1 1000
Progress: 1 1100
Progress: 1 1200
Progress: 1 1300
Progress: 1 1400
Progress: 1 1500
Progress: 1 1600
Progress: 1 